In [3]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import punkt
import string
from tqdm import tqdm as tq
from collections import Counter
import math
import numpy as np

In [4]:

def preprocess(text):
    ### Converting to lowercase
    text = text.lower()
    
    ### Removing Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    ### Converting strings to tokens of individual words in a complete string
    words = word_tokenize(text_p)
    
    ### Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    ### Get the root of the word i.e stemming
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return stemmed


In [13]:
def findquery(query,data,inv,voc):
    try:
        query_prep=preprocess(query)
        query_prep=[voc[i] for i in query_prep]
        doc=set(inv[query_prep[0]])
        
        try:
            for i in query_prep[1:]:
                doc=doc.intersection(inv[i])
        except:
            pass
        
        if doc==set():
            print('is not in the dataframe')
            return None

        return data[data.index.isin(doc)]
    except:
        print('is not in the dataframe')
    

In [14]:
query='american museum'
findquery(query,data,inv,voc)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
59,The Witch House of Salem,"Salem, Massachusetts",3161.0,3957.0,The Salem witchcraft trials took place between...,"The Witch House of Salem,Salem, Massachusetts,...","['Joshua Ward House', 'Statue of Elizabeth Mon...","310 1/2 Essex StreetSalem, Massachusetts, 0197...",42.5215,-70.8989,"['aniashaclay', 'littleredinfiniti', 'cait7911...","June 25, 2009","[""The World's Top 100 Wonders in 2018""]","['The Black Castle', 'Salem Village Parsonage'...",https://www.atlasobscura.com/places/witch-hous...
141,Museum of the Weird,"Austin, Texas",1011.0,2956.0,The dime or dime store museum is by all accoun...,"Museum of the Weird,Austin, Texas, Continuing ...","['Frost Bank Tower', 'Austin Moonlight Towers'...","Museum of the Weird412 E 6th St.Austin, Texas,...",30.2670,-97.7387,"['TheAmazingFace', 'RaspberryLion', 'knoxium',...","April 6, 2011",NaN,NaN,https://www.atlasobscura.com/places/museum-weird
173,National Atomic Testing Museum,"Las Vegas, Nevada",1093.0,2796.0,Las Vegas is an oasis in the desert where no o...,"National Atomic Testing Museum,Las Vegas, Neva...","['High Roller', 'Bugsy Siegel Monument', 'Flam...","755 E Flamingo RdLas Vegas, Nevada, 89119Unite...",36.1141,-115.1484,"['Taterchips', 'paigeelizabeth3660', 'dlc31723...","March 25, 2013",['17 Hidden Las Vegas Spots It Would Be a Sin ...,"['Titan Missile Museum', 'Museum Uranbergbau',...",https://www.atlasobscura.com/places/national-a...
224,Canyons of the Ancients,"Dolores, Colorado",447.0,2541.0,Ripe for quiet reflection and simply awe-inspi...,"Canyons of the Ancients,Dolores, Colorado, A b...","['Mesa Verde National Park', 'Ute Mountain Tri...","27501 CO-184Mesa VerdeDolores, Colorado, 81323...",37.4763,-108.5459,[],"January 21, 2013",['18 Places in Colorado to Discover Local Cult...,"['Acacus Mountains', 'Japanese War Cemetery an...",https://www.atlasobscura.com/places/canyons-of...
320,The Natural Bridge,"Natural Bridge, Virginia",1719.0,2262.0,Often cited as having a place among the great ...,"The Natural Bridge,Natural Bridge, Virginia, A...","['Hidden J.R.R. Tolkien Quote', 'Traveller’s F...","15 Appledore LaneNatural Bridge, Virginia, 245...",37.6282,-79.5447,"['shylilturtle', 'plt4000', 'kjill99', 'lendog...","July 4, 2010",['The Most Wondrous State Parks to Visit Durin...,"['Cueva Clara', '100,000 Soldiers of Trabuc Ca...",https://www.atlasobscura.com/places/the-natura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7064,Wilbur D. May Museum,"Reno, Nevada",131.0,411.0,There is only one place in Nevada where you wi...,"Wilbur D. May Museum,Reno, Nevada, An eccentri...","[""'Jackson'"", 'National Bowling Stadium', 'Top...","1595 N Sierra StReno, NevadaUnited States",39.5457,-119.8240,[],"January 17, 2017",NaN,NaN,https://www.atlasobscura.com/places/wilbur-d-m...
7105,The Liberty Bell Hiding Place,"Allentown, Pennsylvania",146.0,409.0,"In September of 1777, the British defeated Geo...","The Liberty Bell Hiding Place,Allentown, Penns...","['Abandoned Thomas Iron Ruins', ""Leonardo's Ho...","622 Hamilton StAllentown, PennsylvaniaUnited S...",40.6023,-75.4704,[],"February 16, 2017",NaN,NaN,https://www.atlasobscura.com/places/liberty-be...
7147,American Computer Museum,"Bozeman, Montana",124.0,408.0,"“To collect, preserve, interpret, and display ...","American Computer Museum,Bozeman, Montana, See...","['Madison Buffalo Jump State Park', 'Bleu Hors...","2023 Stadium DrSuite 1ABozeman, Montana, 59715...",45.6590,-111.0548,[],"January 30, 2010",['A Road Trip Through the Constantly Evolving ...,"['Computer History Museum', 'Silverball Museum...",https://www.atlasobscura.com/places/american-c...
7154,Akin Free Library,"Pawling, New York",41.0,407.0,"The Akin Free Library, a gracious stone struct...","Akin Free Library,Pawling, New York, The attic...","[""Rainbo

In [7]:
#data=pd.read_csv('final_dataset_2_rm_nan_url.csv')
#inv1=pd.read_csv('inv1.csv')
#vocab=pd.read_csv('vocab.csv')
#invtf=pd.read_csv('invtf.csv')

In [8]:
#data=data.drop(['Unnamed: 0'], axis=1)
#inv1=inv1.drop(['Unnamed: 0'], axis=1)
#vocab=vocab.drop(['Unnamed: 0'], axis=1)
#invtf=invtf.drop(['Unnamed: 0'], axis=1)

In [9]:
#inv1['doc']=list(map(eval,inv1['doc']))
#invtf['doc']=list(map(eval,invtf['doc']))
#vocab['N/n']=list(map(np.log,vocab['N/n'].values))
#voc = dict(vocab[['word','ind']].values)
#inv =  dict(inv1.values)
invtfvoc= dict(invtf.values)

In [10]:
def tfidf(w,preplist,ind):
    return list((preplist.count(w))/len(preplist)*ind)[0]

In [13]:
#### invtf generator
from tqdm import tqdm
import numpy as np
inv2={}
key=vocab['word']
for i,j in tqdm(zip(inv1['ind'],inv1['doc'])):
    w=key[int(i)]   
    inv2[i]={}
    for t in j:
        datas=data_des[data_des['ind']==t]['prep'].values.tolist()[0]
        inv2[i][t]=tfidf(w,datas,vocab['N/n'][i])

0it [00:00, ?it/s]


NameError: name 'data_des' is not defined

In [ ]:
#inv1=pd.DataFrame()
#inv1['ind']=inv2.keys()
#inv1['doc']=inv2.values()
#inv1.to_csv('invtf.csv')

In [15]:
mat= np.zeros((len(voc),7200))
for i in range(len(voc)):
    mat[i,list(invtfvoc[i].keys())]=list(invtfvoc[i].values())

In [22]:
def cosine(query,data,inv,vocab,voc,mat):
    query_prep=preprocess(query)
    
    query_list=[]
    query_val=[]
    for i in query_prep:
        if voc[i] not in query_list:
            query_list.append(voc[i])
            query_val.append(tfidf(i,query_prep,vocab[vocab['word']==i]['N/n']))
        
    datas = findquery(' '.join(query_prep),data,inv,voc)
    indexfind=list(datas.index)
    
    a=[np.sum(query_val*mat[query_list,i])/(np.linalg.norm(query_val)*np.linalg.norm(mat[query_list,i])) for i in indexfind]
    
    datas=datas.assign(Similarity=a)
    
    return datas.sort_values(by=['Similarity'], ascending=False)  

In [54]:
query='american museum zoo'
cosine(query,data,inv,vocab,voc,mat)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,Similarity
1050,Flushing Meadows-Corona Park,"Queens, New York",1373.0,1349.0,"After a long ride from Manhattan, most get off...","Flushing Meadows-Corona Park,Queens, New York,...","['Panorama of the City of New York', 'Fountain...","Kennedy CircleQueens, New York, 11368United St...",40.7462,-73.8445,"['lakupper13', 'cait7911', 'Babs81', 'Corina',...","September 5, 2009","[""A 'No Regrets' Tour With One of Our Top Phot...","[""'Worker and Kolkhoz Woman'"", ""'Jeune Éléphan...",https://www.atlasobscura.com/places/flushing-m...,0.982192
6149,"Martha, the Last Passenger Pigeon","Washington, D.C.",513.0,467.0,When Europeans began settling in the New World...,"Martha, the Last Passenger Pigeon,Washington, ...","['Mummified Bison', 'Carousel on the National ...","10th St. & Constitution Ave. NWWashington, Dis...",38.8912,-77.0260,"['Liana', 'Ancientproof', 'thebodyinthelibrary...","June 6, 2014",['The Ultimate Guide to Bird Monuments'],"['Grip the Raven', 'Anniston Museum', 'Cher Am...",https://www.atlasobscura.com/places/martha-the...,0.937040
1989,Skeletons: Museum of Osteology,"Orlando, Florida",214.0,981.0,Following the success of his original skeleton...,"Skeletons: Museum of Osteology,Orlando, Florid...","['Global Convergence Sculpture', 'Randall Knif...","8441 International Dr.Suite 250Orlando, Florid...",28.4426,-81.4691,[],"May 18, 2017",['Off-Centered Ale Trail: Orlando'],"['Gympie Bone Museum', 'Jurassic Museum of Ast...",https://www.atlasobscura.com/places/skeletons-...,0.809510
5012,Mammoth Cave Wildlife Museum,"Cave City, Kentucky",170.0,550.0,Cave City is a wonderland of kitschy tourist t...,"Mammoth Cave Wildlife Museum,Cave City, Kentuc...","[""Bell's Tavern Ruins"", 'Wigwam Village #2', '...","409 E Happy Valley StCave City, KentuckyUnited...",37.1269,-85.9612,[],"September 4, 2018",NaN,NaN,https://www.atlasobscura.com/places/mammoth-ca...,0.809510
985,Horniman Museum and Gardens,"London, England",743.0,1386.0,London’s Horniman Museum has been showing off ...,"Horniman Museum and Gardens,London, England, A...","[""St Antholin's Spire"", 'Horniman Aquarium', '...","100 London RdForest HillLondon, England, SE23 ...",51.4410,-0.0610,"['Snowdrop68', 'Kaiju', 'Monsieur Mictlan', 'D...","August 1, 2013",NaN,NaN,https://www.atlasobscura.com/places/horniman-m...,0.713007


In [97]:

import plotly.express as px

datas=cosine('american museum',data,inv,vocab,voc,mat)


fig = px.scatter_mapbox(datas,title = 'Place Atlas', lat="placeAlt", lon="placeLong", hover_name="placeName", hover_data=["placeTags", "placeAddress"],
                        color_continuous_scale=px.colors.sequential.Viridis, color="Similarity",size="numPeopleVisited", zoom=4, height=500
                       )

fig.update_layout(
    mapbox_style="open-street-map",
    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.update(layout={'title': {'x': 0.5, 'y': 1}})

fig.show()

In [100]:
import plotly.graph_objects as go
datas=cosine(query+' first',data,inv,vocab,voc,mat)

fig = go.Figure(data=go.Scattermapbox(
        lon = datas['placeLong'],
        lat = datas['placeAlt'],
        text='Name place: '+datas['placeName'],
        
             
        #hover_name="placeName", hover_data=["placeTags", "placeAddress"],
        #color_continuous_scale=px.colors.sequential.Viridis, 
        #color="Similarity",size="numPeopleVisited", zoom=4, height=500
                       
        ))
fig.update_layout(
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        mapbox={
            'center': {'lon': 10, 'lat': 10},
            'style': "open-street-map",
            'center': {'lon': -20, 'lat': -20},
            'zoom': 1})


fig.show()

ValueError: The first argument to the plotly.graph_objs.layout.Legend 
constructor must be a dict or 
an instance of :class:`plotly.graph_objs.layout.Legend`